In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


load the shock capturing bowShock


In [23]:
var database = OpenOrCreateDatabase("bosss_db_levelSets");

In [24]:
var sessions = database.Sessions;
sessions

#0: IBMBowShock_P2	IBMBowShock_p2_xCells40_yCells160_CFLFrac0.1_RK1_s0=1.0E-03_lambdaMax15	5/7/2020 8:33:16 AM	9c45ebf9...


In [25]:
var flds =sessions.Pick(0).Timesteps.Last().Fields.ToList();
flds

index,value
0,levelSet_recon
1,levelSet_recon_prc
2,levelSet_recon_prc_cont
3,rho
4,m0
5,m1
6,rhoE


## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(\gamma -1)(\rho E - \rho e_{kin}),~\rho e_{kin}=\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock ($p_\text{in}=1,\rho_\text{in}=1$) presented, the v-velocity is zero and we have
$$ h_{in} = \rho E_{in}+1$$
We prescribe the inflow conditions by the Mach number $M_{in}=4$. From those the Energy is computed as 
$$ \rho E_{in} = (\rho e_{inner})_\text{in}+(\rho e_{kin})_\text{in}=\frac{p_\text{in}}{\gamma-1}+\frac{\rho_\text{in}}{2}u_\text{in}^2= \frac{1}{\gamma-1}+\frac{1}{2}u_{in}^2$$ 
where $u_{in} = M_{in}\sqrt{\gamma}$. In total we obtain:
$$ h_\text{in}=\rho E_{in}+1=\frac{\gamma}{\gamma-1}+\frac{1}{2}u_{in}^2$$


In [26]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = sessions.Pick(0).Timesteps.Last().GetField("rho");
var enthalpy_exact = (DGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 4.0;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_inflow


14.7

In [27]:
var consVars= flds.GetRange(3,4).ToArray();
consVars

index,value
0,rho
1,m0
2,m1
3,rhoE


load the XDG BowShock to obtain the boundary level set

In [28]:
MatrixAssembler aaa = new MatrixAssembler(12);
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_ConvStudy");
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var db= OpenOrCreateDatabase(dbPath);
var levelSet = (SinglePhaseField) db.Sessions.Pick(0).Timesteps.Last().GetField("levelSet");

Project name is set to 'XESF_BowShock_ConvStudy'.
Default Execution queue is chosen for the database.


get material parameters

In [29]:
var control = XESF.XESFHardCodedControl.XDGBS_Local();
var material = control.GetMaterial();

transform to grid of shock capturing simulation

In [30]:
var lS_onSCGrid = (SinglePhaseField) flds.Pick(0).CloneAs();
lS_onSCGrid.Clear();
lS_onSCGrid.Identification="levelSet";
lS_onSCGrid.ProjectFromForeignGrid(1.0,levelSet);
flds.Add(lS_onSCGrid)

compute entropy and enthalpy field for the shock capturing simulation

In [31]:
using BoSSS.Solution.CompressibleFlowCommon;
double gamma=1.4;
var enthalpy = (SinglePhaseField) consVars[0].CloneAs();
enthalpy.Clear();
enthalpy.Identification="enthalpy";
var enthalpy_err = (SinglePhaseField) consVars[0].CloneAs();
enthalpy_err.Clear();
enthalpy_err.Identification="enthalpy_err";
var entropy = (SinglePhaseField) consVars[0].CloneAs();
entropy.Clear();
entropy.Identification="entropy";
var pressure = (SinglePhaseField) consVars[0].CloneAs();
pressure.Clear();
pressure.Identification="p";
pressure.ProjectFunction(1.0,
    (X, U, j) => U[0] >0 ? new StateVector(U, material).Pressure:0,new CellQuadratureScheme(true,CellMask.GetFullMask(enthalpy.GridDat)),
    consVars);
enthalpy.ProjectFunction(1.0,
    (X, U, j) => U[0] >0 ? new StateVector(U, material).Enthalpy:0 ,new CellQuadratureScheme(true,CellMask.GetFullMask(enthalpy.GridDat)),
    consVars);
enthalpy_err.ProjectFunction(1.0,
    (X, U, j) => U[0] >0 ? new StateVector(U, material).Enthalpy-enthalpy_inflow:0 ,new CellQuadratureScheme(true,CellMask.GetFullMask(enthalpy.GridDat)),
    consVars);
entropy.ProjectFunction(1.0,
    (X, U, j) => U[0] >0 ? new StateVector(U, material).Entropy:0,new CellQuadratureScheme(true,CellMask.GetFullMask(enthalpy.GridDat)),
    consVars);

// enthalpy.GetExtremanlValues(....)
// enthalpy.Coordinates.CheckForNanAndInf(...)
flds.Add(entropy);
flds.Add(enthalpy);
flds.Add(enthalpy_err);
flds.Add(pressure);
flds


index,value
0,levelSet_recon
1,levelSet_recon_prc
2,levelSet_recon_prc_cont
3,rho
4,m0
5,m1
6,rhoE
7,levelSet
8,entropy
9,enthalpy


plot everything

In [32]:
BoSSS.Solution.Application.DeleteOldPlotFiles();
var tp = new Tecplot(enthalpy.GridDat,2);
tp.PlotFields("bowShockSC",0.0,flds);

rm;


Compute the normalized enthalpy error

In [33]:
enthalpy_exact.Clear();
enthalpy_exact.ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2Norm();
var EE = enthalpy_err.L2Norm()/EEN;
EE

0.018287187254112165

## Obtain enthalpy error from XDG solution

In [34]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_ConvStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
var db=BoSSSshell.WorkflowMgm.DefaultDatabase;
db

Project name is set to 'XESF_BowShock_ConvStudy'.
Default Execution queue is chosen for the database.


{ Session Count = 8; Grid Count = 8; Path = \\dc3\userspace\sebastian\cluster\XESF_BowShock_ConvStudy }

In [35]:
var sessions = db.Sessions;
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 3:12:58 PM	13d0abde...
#1: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:20:52 PM	705aac94...
#2: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:43:57 PM	c3bc10a4...
#3: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:16:31 PM	fe883d23...
#4: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:14:05 PM	cab2c050...
#5: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:06 PM	990fe52f...
#6: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:13:56 PM	643f6a58...
#7: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:05 PM	6ec2412f...


In [36]:
//var sessions = database.Sessions.Where(s => s.Timesteps.Count >0); //plot all 
var ses= sessions.Where(s => (s.Name.Contains("iPrb0")  &&s.Name.Contains("5x16")&&s.Name.Contains("iFlx0"))); // the one plotted in the publication
ses


#0: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:05 PM	6ec2412f...


extract last timestep with p=2 and compute compute error

In [37]:
int p=2;
var timesteps= ses.Pick(0).Timesteps;
MatrixAssembler aaa = new MatrixAssembler(3);
var rTSIndex = timesteps.LastIndexWhere(t => t.GetField("rho").Basis.Degree == p);

IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)timesteps.Pick(rTSIndex)).GetInternal();
var enthalpyError = (XDGField) lts.GetField("h_err").Clone();
enthalpyError.L2NormAllSpecies()/EEN


0.0008516470864078319

## compute Entropy

In [38]:
var flds = lts.Fields;
flds


index,type,value
0,BoSSS.Foundation.XDG.XDGField,u0
1,BoSSS.Foundation.XDG.XDGField,u1
2,BoSSS.Foundation.XDG.XDGField,p
3,BoSSS.Foundation.XDG.XDGField,h
4,BoSSS.Foundation.XDG.XDGField,h_err
5,BoSSS.Foundation.XDG.XDGField,Ma
6,BoSSS.Foundation.XDG.LevelSet,levelSet
7,BoSSS.Foundation.XDG.LevelSet,levelSetTwo
8,BoSSS.Foundation.XDG.XDGField,rho
9,BoSSS.Foundation.XDG.XDGField,m0


In [39]:
var consVars= new XDGField[] {(XDGField) flds.ElementAt(8),(XDGField)flds.ElementAt(9),(XDGField)flds.ElementAt(10),(XDGField)flds.ElementAt(11)};
consVars

index,value
0,rho
1,m0
2,m1
3,rhoE


In [40]:
using BoSSS.Solution.CompressibleFlowCommon;
double gamma=1.4;
var entropy =  consVars[0].CloneAs();
entropy.Clear();
entropy.Identification="entropy";
entropy.ProjectFunctionXDG(1.0,
    (X, U, j) => U[0] >0 ? new StateVector(U, material).Entropy:0,
    consVars);

// enthalpy.GetExtremanlValues(....)
// enthalpy.Coordinates.CheckForNanAndInf(...)

